# AMPLPY: Google Colab & Kaggle Template

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplpy/blob/master/notebooks/colab.ipynb)

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplpy/blob/master/notebooks/colab.ipynb)

Documentation: http://amplpy.readthedocs.io

GitHub Repository: https://github.com/ampl/amplpy

PyPI Repository: https://pypi.python.org/pypi/amplpy

Jupyter Notebooks: https://github.com/ampl/amplpy/tree/master/notebooks

### Setup

In [1]:
!pip install -q amplpy ampltools

### Google Colab & Kaggle interagration

In [2]:
MODULES=['ampl', 'coin']
from ampltools import cloud_platform_name, ampl_notebook
from amplpy import AMPL, register_magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = ampl_notebook(modules=MODULES) # Install AMPL and use it
register_magics(ampl_object=ampl) # Evaluate %%ampl_eval cells with ampl.eval()

### Use `%%ampl_eval` to evaluate AMPL commands

In [3]:
%%ampl_eval
option version;

option version 'AMPL Version 20220219 (Darwin-19.6.0, 64-bit)\
Demo license with maintenance expiring 20240131.\
Using license file "/Users/fdabrandao/bin/ampl.macos64/ampl.lic".\
';


### Use %%writeifile to create files

In [4]:
%%writefile cut2.mod

  problem Cutting_Opt;
# ----------------------------------------

param nPAT integer >= 0, default 0;
param roll_width;

set PATTERNS = 1..nPAT;
set WIDTHS;

param orders {WIDTHS} > 0;
param nbr {WIDTHS,PATTERNS} integer >= 0;

   check {j in PATTERNS}: sum {i in WIDTHS} i * nbr[i,j] <= roll_width;

var Cut {PATTERNS} integer >= 0;

minimize Number: sum {j in PATTERNS} Cut[j];

subject to Fill {i in WIDTHS}:
   sum {j in PATTERNS} nbr[i,j] * Cut[j] >= orders[i];


  problem Pattern_Gen;
# ----------------------------------------

param price {WIDTHS} default 0;

var Use {WIDTHS} integer >= 0;

minimize Reduced_Cost:
   1 - sum {i in WIDTHS} price[i] * Use[i];

subject to Width_Limit:
   sum {i in WIDTHS} i * Use[i] <= roll_width;

Overwriting cut2.mod


In [5]:
%%writefile cut.dat
data;

param roll_width := 110 ;

param: WIDTHS: orders :=
          20     48
          45     35
          50     24
          55     10
          75      8  ;

Overwriting cut.dat


In [6]:
%%writefile cut2.run
# ----------------------------------------
# GILMORE-GOMORY METHOD FOR
# CUTTING STOCK PROBLEM
# ----------------------------------------

option solver cbc;
option solution_round 6;

model cut2.mod;
data cut.dat;

problem Cutting_Opt;
   option relax_integrality 1;
   option presolve 0;

problem Pattern_Gen;
   option relax_integrality 0;
   option presolve 1;

let nPAT := 0;

for {i in WIDTHS} {
   let nPAT := nPAT + 1;
   let nbr[i,nPAT] := floor (roll_width/i);
   let {i2 in WIDTHS: i2 <> i} nbr[i2,nPAT] := 0;
   };

repeat {
   solve Cutting_Opt;

   let {i in WIDTHS} price[i] := Fill[i].dual;

   solve Pattern_Gen;

   if Reduced_Cost < -0.00001 then {
      let nPAT := nPAT + 1;
      let {i in WIDTHS} nbr[i,nPAT] := Use[i];
      }
   else break;
   };

display nbr; 
display Cut;

option Cutting_Opt.relax_integrality 0;
option Cutting_Opt.presolve 10;
solve Cutting_Opt;

display Cut;

Overwriting cut2.run


### Use `%%ampl_eval` to run the script cut2.run

In [7]:
%%ampl_eval
commands cut2.run;

CBC 2.10.5: CBC 2.10.5 optimal, objective 52.1
0 iterations
CBC 2.10.5: CBC 2.10.5 optimal, objective -0.2
0 nodes, 3 iterations, 0.005544 seconds
CBC 2.10.5: CBC 2.10.5 optimal, objective 50.5
1 iterations
CBC 2.10.5: CBC 2.10.5 optimal, objective -0.2
0 nodes, 0 iterations, 0.002618 seconds
CBC 2.10.5: CBC 2.10.5 optimal, objective 47
1 iterations
CBC 2.10.5: CBC 2.10.5 optimal, objective -0.1
0 nodes, 0 iterations, 0.003455 seconds
CBC 2.10.5: CBC 2.10.5 optimal, objective 46.25
1 iterations
CBC 2.10.5: CBC 2.10.5 optimal, objective 0
0 nodes, 6 iterations, 0.004011 seconds
nbr [*,*] (tr)
:  20  45  50  55  75    :=
1   5   0   0   0   0
2   0   2   0   0   0
3   0   0   2   0   0
4   0   0   0   2   0
5   0   0   0   0   1
6   1   0   0   0   1
7   1   2   0   0   0
8   3   0   1   0   0
;

Cut [*] :=
1   0
2   0
3   8.25
4   5
5   0
6   8
7  17.5
8   7.5
;

CBC 2.10.5: CBC 2